```
ME72: Maestría en Métodos Cuantitativos para la Gestión y Análisis de Datos
M72109: Analisis de datos no estructurados
Universidad de Buenos Aires - Facultad de Ciencias Economicas (UBA-FCE)
Año: 2023
Profesor: Facundo Santiago
```

# Solución: ¿Qué tan memorable es un video?

Este notebook les permitirá tener a disposición todos los recursos que se vieron en durante la materia. Todos los fragmentos de código se descargaran utilizando la sección preparación del ambiente.

## Preparación del ambiente

### Sets de datos

In [1]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/ground_truth.csv --directory-prefix ./Data/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/Data/Features/caption_features.csv --directory-prefix ./Data/Features/ --quiet

### Librerías de codigo que se utilizan durante el curso

In [2]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/Desafio/solucion_base.txt --quiet
!pip install -r solucion_base.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 21.6 MB/s eta 0:00:00
ERROR: pip's dependency resolv

#### NLP

In [6]:
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/datasets.py --directory-prefix ./m72109/nlp/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/explanation.py --directory-prefix ./m72109/nlp/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/normalization.py --directory-prefix ./m72109/nlp/ --quiet
!wget -N https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/transformation.py --directory-prefix ./m72109/nlp/ --quiet
!wget https://santiagxf.blob.core.windows.net/public/Word2Vec/model-en.bin --directory-prefix ./Models/Word2Vec --quiet

### Imports

In [7]:
import pandas as pd
import numpy as np
import os
import torch
import transformers
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from m72109.nlp.normalization import TextNormalizer, TweetTextNormalizer
from m72109.nlp.transformation import Word2VecVectorizer, PadSequenceTransformer, split_text_with_context
from m72109.nlp.explanation import get_explanation_from_grads, locate_token

## Solución

Cuentan con 2 tipos de anotaciones para cada uno de los fragmentos de video disponibles:
 - **memorability_score:** Representa el puntaje de memorabilidad de la secuencia en particular, desde 0 a 1. Valores más grandes son mejores.
 - **memorable:** Variable categórica que representa si un video es memorable o no. Un video con `memorability_score` superior a `0.5` es marcado como memorable (`1`), sino es marcado como no memorable (`0`)

In [8]:
labels = pd.read_csv('Data/ground_truth.csv')

In [9]:
labels.head(2)

,movie_name,start(sec),end(sec),sequence_name,Neutral (1)_Typical (0),nb_annotations,memorability_score,memorabable
0,127 hours,2000,2010,127_hours_2000_2010_1,0,5,1.0,1
1,127 hours,2182,2192,127_hours_2182_2192_5,1,8,0.0,0


Las siguientes variables contienen la información preprocesada:

In [10]:
captions = pd.read_csv('Data/Features/caption_features.csv')

In [ ]:
# completar solución